In [1]:
import requests, json, jwt
import base64
from ipywidgets import widgets, Layout
from IPython.display import display
from IPython.display import HTML

requests.packages.urllib3.disable_warnings()

In [2]:
tenant = "amsidcstrial20"
domain = "identity.oraclecloud.com"
port = "443"

baseurl = 'https://' + tenant + '.' + domain + ':' + port
configurl =  baseurl + '/.well-known/idcs-configuration'
response = requests.get(configurl, verify=False).text
config = json.loads(response)

In [3]:
clientId = "017126b0d525454aa846aebff5aa05b6"
clientSecret = "d543567a-e3dc-4fb0-8a94-15f32c35825d"

In [4]:
userName = "indranil.jha@oracle.com"
userPwd = "Oracle123"

In [6]:
tokenurl = config["openid-configuration"]["token_endpoint"]

grant_type = "client_credentials"
scope = "urn:opc:idm:__myscopes__"

basicauthHeader = base64.b64encode(bytes(clientId + ":" + clientSecret)) 
reqdata = {'grant_type': grant_type, 'scope' : scope}
reqheaders = {'Authorization': 'Basic ' + basicauthHeader.decode("utf-8"), 'content-type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data = reqdata, headers = reqheaders, verify=False)
accesstoken = response.json()["access_token"]
headers = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
accesstokenD = widgets.Accordion(children=[widgets.Textarea(value=accesstoken, layout=Layout(width='100%', height='400px'))])
accesstokenD.set_title(0, 'Access Token')
accesstokenD.selected_index = None
display(accesstokenD)

A Jupyter Widget

In [7]:
callurl1 = baseurl + "/sso/v1/sdk/authenticate"
print(callurl1)
response = requests.get(callurl1, headers = headers, verify=False)
print(response.text)
requestState = json.loads(response.text)["requestState"]

https://amsidcstrial20.identity.oraclecloud.com:443/sso/v1/sdk/authenticate
<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html;charset=UTF-8"><TITLE>Unauthorized</TITLE></HEAD>
<BODY><H1>Unauthorized</H1>
Proper authorization is required for this area. Either your browser does not perform authorization, or your authorization has failed.
</BODY></HTML>


ValueError: No JSON object could be decoded

In [ ]:
callurl2 = baseurl + "/sso/v1/sdk/authenticate"
payload2 = {
            "op":"credSubmit",
            "credentials":{
                "username": userName,
                "password": userPwd
            },
            "requestState":requestState
           }
response2 = requests.post(callurl2, headers = headers, data =json.dumps(payload2),  verify=False)
authtoken = json.loads(response2.text)["authnToken"]
print(response2.text)
print(authtoken)

In [ ]:
jwtoken = jwt.decode(authtoken, verify=False)

jwtokenD = widgets.Accordion(children=[widgets.Textarea(value=json.dumps(jwtoken, indent=1), layout=Layout(width='100%', height='600px'))])
jwtokenD.set_title(0, 'Inspect Token')
jwtokenD.selected_index = None
display(jwtokenD)
